# IMPLEMENTATION OF THE FINAL MODEL USING A WEB APP CAN BE FOUND IN THE FOLLOWING LINK http://bit.ly/HenonAminaAmharicSmS

In [1]:
# This program detects if an SMS is Spam or not
import pandas as pd 
import numpy as np 
import nltk
from nltk.corpus import stopwords
import string 

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
#load the data 
dataframe = pd.read_excel("togezer.xlsx")

In [4]:
#print some of the data 
print(dataframe.head(5))

                                                 sms label
0           አይ እዛ ሚማር አይመስለኝም . እዛ አካባቢ ነው ግን የሚኖረው    ham
1    ወንድሜ እንኳ አያናግረኝም።  እንደ ኤች አይ ቪ በሽተኛ አድርገው ይይዙኛል   ham
2                                                NaN   ham
3  በቅርቡ ወደ ቤት እመለሳለሁ እናም ዛሬ ስለዚህ ነገር ድጋሚ ማውራት አልፈ...   ham
4  ለዚህ እስትንፋስ ላመሰግናችሁ ትክክለኛዎቹን ቃላት እየፈለግኩ ነው ፡፡ እ...   ham


In [5]:
#get column name 
dataframe.columns

Index(['sms', 'label'], dtype='object')

In [6]:
#get column shape 
dataframe.shape

(1002, 2)

In [7]:
# chceck the new shape
dataframe.info()
dataframe.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 2 columns):
sms      993 non-null object
label    1002 non-null object
dtypes: object(2)
memory usage: 15.8+ KB


,sms,label
count,993,1002
unique,979,2
top,አሁን ብቻ 1 ጊዜ ከላኩ በቂ ነው!ያውም በነፃ!የመቄዶንያ አባል ይሁኑ!1...,spam
freq,2,544


In [8]:
# show the number of missing data for each coloum(NAN, NAN, na)
dataframe.isnull().sum()

sms      9
label    0
dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
encode=LabelEncoder()
label_new=encode.fit_transform(dataframe['label'])
label_new

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
import re
import itertools
def clean_data(sentence):
    ## removing web links
    s = [ re.sub(r'http\S+', '', sentence.lower())]
    s = [''.join(''.join(s)[:2] for _, s in itertools.groupby(s[0]))]

    ## removing punctuations from the code 
    s = [remove_punctuations(s[0])]
    s = [ re.sub('(ሃ|ኃ|ሓ|ሀ|ሐ|ኀ)','ሀ',s[0])]
    s = [ re.sub('(ሁ|ኁ|ሑ)','ሁ',s[0])]
    s = [ re.sub('(ሂ|ኂ|ሒ)','ሂ',s[0])]
    s = [ re.sub('(ሄ|ኄ|ሔ)','ሄ',s[0])]
    s = [ re.sub('(ህ|ኅ|ሕ)','ህ',s[0])]
    s = [ re.sub('(ሆ|ኆ|ሖ)','ሆ',s[0])]
    
    s = [ re.sub('(ሰ|ሠ)','ሰ',s[0])]
    s = [ re.sub('(ሱ|ሡ)','ሱ',s[0])]
    s = [ re.sub('(ሲ|ሢ)','ሲ',s[0])]
    s = [ re.sub('(ሳ|ሣ)','ሳ',s[0])]
    s = [ re.sub('(ሴ|ሤ)','ሴ',s[0])]
    s = [ re.sub('(ስ|ሥ)','ስ',s[0])]
    s = [ re.sub('(ሶ|ሦ)','ሶ',s[0])]
    
    s = [ re.sub('(አ|ኣ|ዐ|ዓ)','አ',s[0])]
    s = [ re.sub('(ኡ|ዑ)','ኡ',s[0])]
    s = [ re.sub('(ኢ|ዒ)','ኢ',s[0])]
    s = [ re.sub('(ኤ|ዔ)','ኤ',s[0])]
    s = [ re.sub('(እ|ዕ)','እ',s[0])]
    s = [ re.sub('(ኦ|ዖ)','ኦ',s[0])]
    
    s = [ re.sub('(ጸ|ፀ)','ፀ',s[0])]
    s = [ re.sub('(ጹ|ፁ)','ፁ',s[0])]
    s = [ re.sub('(ጺ|ፂ)','ፂ',s[0])]
    s = [ re.sub('(ጻ|ፃ)','ፃ',s[0])]
    s = [ re.sub('(ጼ|ፄ)','ፄ',s[0])]
    s = [ re.sub('(ጽ|ፅ)','ፅ',s[0])]
    s = [ re.sub('(ጾ|ፆ)','ፆ',s[0])]
    return s[0]

In [11]:
def remove_punctuations(my_str):
    punctuations = '''!()-[]{};:'"\,./?@#$%^&@*_~።፣፤፥?፤፡'''
    no_punct = ""
    for char in my_str:
       if char not in punctuations:
           no_punct = no_punct + char
    return no_punct

In [12]:
## Sample Sentence to be cleaned
sentence="የጠቅላላ ዕ፡ውቀት ክህሎቶን ከፍ ለማድረግ እንዲሁምሓ  የፋይናኀንስ እና የቢዝነስ ምክር ለማግኘት ወደ 6642 Ok ብለው በነጻ ይላኩ::"

## Using clean_data function
clean_data(sentence)



'የጠቅላላ እውቀት ክህሎቶን ከፍ ለማድረግ እንዲሁምሀ  የፋይናሀንስ እና የቢዝነስ ምክር ለማግኘት ወደ 6642 ok ብለው በነፃ ይላኩ'

In [13]:
for index in range(0,len(dataframe["sms"])):
    dataframe.loc[index,"sms"] = clean_data(str(dataframe["sms"].iloc[index]))

In [14]:
dataframe['sms']

0                 አይ እዛ ሚማር አይመስለኝም  እዛ አካባቢ ነው ግን የሚኖረው 
1          ወንድሜ እንኳ አያናግረኝም  እንደ ኤች አይ ቪ በሽተኛ አድርገው ይይዙኛል
2                                                     nan
3       በቅርቡ ወደ ቤት እመለሳለሁ እናም ዛሬ ስለዚህ ነገር ድጋሚ ማውራት አልፈ...
4       ለዚህ እስትንፋስ ላመሰግናችሁ ትክክለኛዎቹን ቃላት እየፈለግኩ ነው  እኔ ...
                              ...                        
997                          ቀላል አደል ሴን ተመርጧል ማለት ጥሩ ነው  
998                                 የካቲት ላይ ፈተና መውሰድ አለብኝ
999     አዎ ብታደርገው መክራለው በኤቲኤምህ መመዝግብ የምትችል ይመስለኛል አርግጠ...
1000                                    ጣጣ የለውም ጊዜሽን ውሰጅ 
1001                                       ማውራት ፈልጋለው ደውል
Name: sms, Length: 1002, dtype: object

In [15]:
#s=dataframe["sms"].iloc[0]
#tokens = nltk.tokenize.word_tokenize(s)
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
tfvect=TfidfVectorizer(max_df=0.7)
trans=tfvect.fit_transform(dataframe['sms'])
messages_bow=pd.DataFrame(trans.toarray(),columns=tfvect.get_feature_names())
messages_bow

,00,0412,0512,058100181178,0912603853,0921452394,10,100,1000,10000,...,ፒጵፒጵፒፒጵበ,ፒጵፒጵፒፒጵየእርሶ,ፓምፕ,ፓርቲ,ፓኪ,ፕላዛ,ፕራቡ,ፕሮጀክቶች,ፕሮግራም,ፖስታ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#split the data into 80% training and 20% testing 
X_train, X_test, y_train, y_test = train_test_split(dataframe['sms'], label_new, test_size=0.20 )

In [17]:
X_train=tfvect.fit_transform(X_train)
_test=tfvect.transform(X_test)

In [18]:
#create and train the naive bayes classifier
classifier = MultinomialNB().fit(X_train,y_train)

In [19]:
#print the pridiction
prediction=classifier.predict(_test)
prediction

array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0])

In [20]:
#print the actual values
y_test

array([0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 0])

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix
score=accuracy_score(y_test,prediction)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.04%


In [25]:
 #Evaluate the model on the training data set 
print('Confusion Matrix: \n', confusion_matrix(y_test,prediction))
print()
print('Accuraccy: ', accuracy_score(y_test, prediction))

Confusion Matrix: 
 [[ 82  16]
 [  0 103]]

Accuraccy:  0.9203980099502488


In [32]:
from sklearn.linear_model import LogisticRegression
alg=LogisticRegression()
alg.fit(X_train,y_train)
pred=alg.predict(_test)

In [33]:
score=accuracy_score(y_test,pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 96.02%


In [34]:
 #Evaluate the model on the training data set 
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuraccy: ', accuracy_score(y_test, pred))

Confusion Matrix: 
 [[94  4]
 [ 4 99]]

Accuraccy:  0.9601990049751243


In [40]:
import anvil.server
anvil.server.connect("MVZAD6IJSYGRWFK3JVXOMZ2H-PUBNCWAUKVX7YP7P")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK
Reconnecting Anvil Uplink...


In [41]:
import pickle
import anvil.server
#saving model
pickle.dump(alg, open(r'model_new9.pickle', 'wb'))
@anvil.server.callable
def predict_model(q):
    q=tfvect.transform([q])
    Modelr = pickle.load(open(r'model_new9.pickle', 'rb'))
    predictions=Modelr.predict(q)
    return(predictions)